#Demo de Mars Explorer simulando la arquitectura Subsumption.
- Objetivo : Explorar un planeta distante y en particular recolectar muestras de un cierto tipo de roca.
- Problema: La localización de las rocas no se conoce, pero se sabe que ellas tienden a estar agrupadas en regiones cercanas

In [1]:
#@title Cargar Librerías

from random import randint
import copy

print("Librerías cargadas.")

Librerías cargadas.


In [6]:
#@title Definir Mapa al Azar
limiteMaximo = 10 #@param{"type":"integer"}
cantObstaculos = 30 #@param{"type":"integer"}
cantMuestras = 15 #@param{"type":"integer"}
if limiteMaximo<10:
  limiteMaximo = 10
if cantObstaculos<1:
  cantObstaculos = 1
if cantMuestras<1:
  cantMuestras = 1

# genera obstaculos
ObstaculosPos = []
for _ in range(30):
  ObstaculosPos.append(  [ randint(0, limiteMaximo),  randint(0, limiteMaximo) ] )
print("Obstaculos en ", ObstaculosPos)

# genera muestras
MuestrasPos = []
for _ in range(15):
  auxPos = [ randint(0, limiteMaximo),  randint(0, limiteMaximo) ]
  if auxPos not in ObstaculosPos:
    MuestrasPos.append( auxPos)
print("Muestras en ", MuestrasPos )

Obstaculos en  [[10, 2], [9, 9], [1, 10], [0, 10], [6, 10], [1, 10], [8, 4], [10, 9], [9, 6], [7, 2], [9, 6], [10, 2], [9, 9], [5, 7], [9, 8], [0, 10], [3, 6], [7, 6], [10, 5], [6, 1], [2, 8], [9, 4], [10, 9], [7, 4], [7, 0], [5, 8], [3, 7], [2, 3], [10, 5], [9, 8]]
Muestras en  [[8, 7], [7, 10], [2, 6], [1, 7], [9, 3], [5, 3], [8, 8], [2, 0], [1, 7], [1, 4], [10, 7], [0, 2]]


In [3]:
#@title Definir clase MarsExplorer

POS_NADA = 0
POS_BASE = 1
POS_MUESTRA = 2
POS_OBSTACULO = -1

class MarsExplorer(object):

    def __init__(self, posInicial, limiteMaximo, obstaculosPos, muestrasPos):
      self.posActual = copy.deepcopy(posInicial)
      self.posBase = copy.deepcopy(posInicial)
      self.llevaMuestra = False
      self._limiteMaximo = limiteMaximo
      self._obstaculosPos = obstaculosPos
      self._muestrasPos = muestrasPos

    def determinaMov(self, regresaNave=False):
      # nueva posición
      nPos = copy.deepcopy(self.posActual)
      if regresaNave:
          # si lleva muestra debe volver a la nave
          # sino es al azar
          for i in range(len(nPos)):
            if (nPos[i]>0) and (nPos[i]<self._limiteMaximo):
              if self.posBase[i]>nPos[i]:
                nPos[i] += 1
              elif self.posBase[i]<nPos[i]:
                nPos[i] -= 1
      else:
          # cambia posición al azar
          for i in range(len(nPos)):
            if (nPos[i]==0):
               nPos[i] += 1
            elif (nPos[i]==self._limiteMaximo):
              nPos[i] -= 1
            else:
              rnd = randint(-1, 1)
              if (i == 0) and (rnd==0):
                nPos[i] += 1
              else:
                nPos[i] += rnd
      # devuelve nueva posición
      return nPos

    def distanciaBase(self, pos):
      # calcula distancia entre dos puntos
      dist = 0.0
      for p, b in zip(pos, self.posBase):
        dist += (p - b)**2
      return dist**0.5

    def evaluarPosicion(self, nPos):
      # determina si hay algo en la posicion usando random
      if self.distanciaBase(nPos)<1:
        # está en base
        return POS_BASE
      elif nPos in self._obstaculosPos:
        # hay obstaculo
        return POS_OBSTACULO
      elif nPos in self._muestrasPos:
        # hay muestra
        return POS_MUESTRA
      else:
        return POS_NADA

    def analizarMuestra(self,pos):
      # determina si es interesante
      # si la 1er posición es par
      return (randint(1, 2)==1)

    def tomarMuestra(self):
      # toma la muestra
      self.llevaMuestra = True

    def dejarMuestra(self):
      # deja la muestra
      self.llevaMuestra = False

    def reglasC0(self, it, nPos, resPos):
        ##C0 - Evitar obstáculos
        if resPos == POS_OBSTACULO:
          # descarta posición (no la actualiza)
          # determina nueva posición al azar
          azarPos = self.determinaMov(False)
          print(it, "-- C0: Evita OBSTACULO en", nPos, "->", azarPos)
          return True, azarPos
        else:
          return False, nPos

    def reglasC1(self, it, nPos, resPos):
        ##C1 - Dejar muestras en la nave
        if self.llevaMuestra and (resPos == POS_BASE):
          print(it, "-- C1: Deja MUESTRA en NAVE en", self.posBase, "")
          # deja muestra
          self.dejarMuestra()
          # actualiza posición
          self.posActual = copy.deepcopy(self.posBase)
          return True, self.posActual
        else:
          return False, self.posActual

    def reglasC2(self, it, nPos, resPos):
        ##C2 - Llevar muestras a la nave
        if self.llevaMuestra and (resPos != POS_BASE):
          # determina nueva posición para regresar nave
          regPos = self.determinaMov(True)
          print(it, "-- C2: Llevando MUESTRA a NAVE por",nPos, "->", regPos)
          # actualiza posición
          self.posActual = regPos
          return True, regPos
        else:
          return False, nPos


    def reglasC3y4(self, it, nPos, resPos):
        ##C4 - Identificar muestras
        if not(self.llevaMuestra) and (resPos == POS_MUESTRA):
          # analiza la muestra
          detectaMuestraInt = self.analizarMuestra(nPos)
          print(it, "-- C4: Identifica MUESTRA ( interesante=", detectaMuestraInt, ") en ", nPos, "")
          # si interesante la toma, sino la descarta
          if detectaMuestraInt:
            ##C3 - Recolectar muestras
            print(it, "-- C3: Recolecta MUESTRA en", nPos, "")
            # toma muestra
            self.tomarMuestra()
          # actualiza posición
          self.posActual = nPos
          return True, nPos
        else:
          return False, nPos

    def reglasC5(self, it, nPos, resPos):
        ##C5 - Deambular (moverse al azar)
        # determina nueva posición al azar
        azarPos = self.determinaMov(False)
        print(it, "C5- DEAMBULAR por", nPos, "->", azarPos)
        # actualiza posición
        self.posActual = azarPos
        return True, azarPos


    def ejecutar(self, N):
      # define las regla a ejecutar
      reglaComportamiento = [self.reglasC0, self.reglasC1, self.reglasC2, self.reglasC3y4, self.reglasC5]
      # ejecuta
      print(0, "-- Comienza en NAVE ",  self.posBase, "")
      nPos = self.posBase
      for i in range(1, N+1):
        # analiza nueva posición
        resPos = self.evaluarPosicion(nPos)
        # ejecuta la regla que corresponda
        for regla in reglaComportamiento:
          resRegla, nPos = regla(i, nPos, resPos)
          if resRegla:
            break


print("clase MarsExplorer definida.")

clase MarsExplorer definida.


In [7]:
#@title Indicar cantidad de ciclos de Ejecución (N)
N = 100 #@param {"type":"integer"}
if N<5:
  N=5

# ejecuta
posIni = [5, 5]
me = MarsExplorer(posIni, limiteMaximo,
                  ObstaculosPos,
                  MuestrasPos)
me.ejecutar(N)


0 -- Comienza en NAVE  [5, 5] 
1 C5- DEAMBULAR por [5, 5] -> [4, 4]
2 C5- DEAMBULAR por [4, 4] -> [5, 3]
3 -- C4: Identifica MUESTRA ( interesante= False ) en  [5, 3] 
4 -- C4: Identifica MUESTRA ( interesante= False ) en  [5, 3] 
5 -- C4: Identifica MUESTRA ( interesante= True ) en  [5, 3] 
5 -- C3: Recolecta MUESTRA en [5, 3] 
6 -- C2: Llevando MUESTRA a NAVE por [5, 3] -> [5, 4]
7 -- C2: Llevando MUESTRA a NAVE por [5, 4] -> [5, 5]
8 -- C1: Deja MUESTRA en NAVE en [5, 5] 
9 C5- DEAMBULAR por [5, 5] -> [6, 5]
10 C5- DEAMBULAR por [6, 5] -> [5, 4]
11 C5- DEAMBULAR por [5, 4] -> [6, 5]
12 C5- DEAMBULAR por [6, 5] -> [5, 4]
13 C5- DEAMBULAR por [5, 4] -> [6, 5]
14 C5- DEAMBULAR por [6, 5] -> [5, 4]
15 C5- DEAMBULAR por [5, 4] -> [4, 4]
16 C5- DEAMBULAR por [4, 4] -> [5, 5]
17 C5- DEAMBULAR por [5, 5] -> [4, 4]
18 C5- DEAMBULAR por [4, 4] -> [5, 5]
19 C5- DEAMBULAR por [5, 5] -> [6, 5]
20 C5- DEAMBULAR por [6, 5] -> [7, 6]
21 -- C0: Evita OBSTACULO en [7, 6] -> [8, 7]
22 -- C4: Identific